# Tavily Extract

[Tavily](https://tavily.com) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed. Tavily offers an [Extract](https://docs.tavily.com/api-reference/endpoint/extract) endpoint that can be used to extract content from a URLs.

## Overview

### Integration details
| Class | Package | Serializable | [JS support](https://js.langchain.com/docs/integrations/tools/tavily_search) |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| [TavilyExtract](https://python.langchain.com/api_reference/community/tools/langchain_community.tools.tavily_extract.tool.TavilyExtract.html) | [langchain-community](https://python.langchain.com/api_reference/community/index.html) | ❌ | ❌ |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-community?style=flat-square&label=%20) |

### Tool features
| [Returns artifact](/docs/how_to/tool_artifacts/) | Native async | Return data | Pricing |
| :---: | :---: | :---: | :---: |
| ❌ | ✅ | Title, URL, content, answer | 1,000 free searches / month | 


## Setup

The integration lives in the `langchain-community` package. We also need to install the `tavily-python` package.

In [13]:
%pip install -qU "langchain-community>=0.3.18" tavily-python


[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Credentials

We also need to set our Tavily API key. You can get an API key by visiting [this site](https://app.tavily.com/sign-in) and creating an account.

In [3]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")


It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [3]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Instantiation

Here we show how to instantiate an instance of the Tavily extract tool. The tool accepts various parameters, including:

In [4]:
from langchain_community.tools import TavilyExtract

tool = TavilyExtract(
    extract_depth="basic",
    include_images=True,
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

## Invocation

### [Invoke directly with args](/docs/concepts/tools)

The `TavilyExtract` tool accepts the following arguments:

- `urls` (required): A list of URLs to extract content from.
- `extract_depth` (optional): The depth of the extraction.
- `include_images` (optional): Whether to include images in the extraction.

NOTE: The optional arguments are available for ReAct agents to dynamically set, if you set a argument during instantiation and then Invoke the tool with a different value, the tool will use the value you passed during invokation. To read more about ReAct agents, check out the [ReAct agent](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/) documentation.


In [11]:
tool.invoke({"urls": ["https://en.wikipedia.org/wiki/Lionel_Messi"]})


{'results': [{'url': 'https://en.wikipedia.org/wiki/Lionel_Messi',
   'raw_content': 'Contents\n\nLionel Messi\n\n\n\nMessi withArgentinaat the2022 FIFA World Cup\nPersonal information\nFull name | Lionel Andrés Messi[1]\nDate of birth | (1987-06-24)24 June 1987(age\xa037)[1]\nPlace of birth | Rosario, Argentina\nHeight | 1.70\xa0m (5\xa0ft 7\xa0in)[1]\nPosition(s) | Forward\nTeam information\nCurrent team | Inter Miami\nNumber | 10\nYouth career\n1992–1995 | Grandoli\n1995–2000 | Newell\'s Old Boys\n2000–2004 | Barcelona\nSenior career*\nYears | Team | Apps | (Gls)\n2003–2004 | Barcelona C | 10 | (5)\n2004–2005 | Barcelona B | 22 | (6)\n2004–2021 | Barcelona | 520 | (474)\n2021–2023 | Paris Saint-Germain | 58 | (22)\n2023– | Inter Miami | 26 | (21)\nInternational career‡\n2004–2005 | Argentina U20 | 18 | (14)\n2008 | Argentina U23 | 5[α] | (2)\n2005– | Argentina | 191 | (112)\nMedal recordMen\'sfootballRepresentingArgentinaFIFA World Cup2022 Qatar2014 BrazilCopa América2021 Brazil2024

### [Invoke with ToolCall](/docs/concepts/tools)

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:

In [12]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"urls": ["https://en.wikipedia.org/wiki/Lionel_Messi"]},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg.content[:400])

{"results": [{"url": "https://en.wikipedia.org/wiki/Lionel_Messi", "raw_content": "Contents\n\nLionel Messi\n\n\n\nMessi withArgentinaat the2022 FIFA World Cup\nPersonal information\nFull name | Lionel Andrés Messi[1]\nDate of birth | (1987-06-24)24 June 1987(age 37)[1]\nPlace of birth | Rosario, Argentina\nHeight | 1.70 m (5 ft 7 in)[1]\nPosition(s) | Forward\nTeam information\nCurrent team | Int


## API reference

For detailed documentation of all TavilySearchResults features and configurations head to the API reference: https://python.langchain.com/api_reference/community/tools/langchain_community.tools.tavily_extract.tool.TavilyExtract.html